In [1]:
import sys
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import random
import time
import torch.utils.data

sys.path.append("../src/")
import data_reader as dr
from evaluate import Evaluation
from lstm import LSTM
from loss_function import loss_function
from loss_function import cs
from meter import AUCMeter

In [2]:
corpus_path = "../data_Android/corpus.tsv.gz"

corpus = dr.read_corpus(corpus_path)

embedding_path = "../data/glove.combined.300d.txt.gz"
embedding_tensor, word_to_indx = dr.getEmbeddingTensor(embedding_path)

ids_corpus = dr.map_corpus(corpus, word_to_indx, kernel_width = 1)

In [3]:
train_corpus_path = "../data/text_tokenized.txt.gz"
train_corpus = dr.read_corpus(train_corpus_path)
train_ids_corpus = dr.map_corpus(train_corpus, word_to_indx, kernel_width = 1)
train_path = "../data/train_random.txt"
train = dr.read_annotations(train_path)
train_ex = dr.create_android_train_set(train_ids_corpus, ids_corpus, train)

In [4]:
eval_path = "../data_Android/dev_android.txt"
eval_anno = dr.read_annotations(eval_path, K_neg = -1, prune_pos_cnt = -1, ignore_dup = True)
eval_set = dr.create_dev_set(ids_corpus, eval_anno)

In [6]:
from transfer_model import Encoder
from transfer_model import DomainClassifier

encoder = Encoder(200, embedding_tensor, 0.2)
dc = DomainClassifier(200, 300, 150)

In [7]:
import torch.utils.data

def train_model(train_data, dev_data, encoder, dc):
    encoder.cuda()
    dc.cuda()
    optimizer_encoder = torch.optim.Adam(filter(lambda p: p.requires_grad, encoder.parameters()), lr=0.0001)
    optimizer_dc = torch.optim.Adam(filter(lambda p: p.requires_grad, dc.parameters()), lr=-0.0001)

    lasttime = time.time()
    for epoch in range(1, 31):
        print("-------------\nEpoch {}:\n".format(epoch))

        loss = run_epoch(train_data, True, encoder, dc, optimizer_encoder, optimizer_dc, 40)
        (reg_loss, bceloss) = loss
        print('Train loss: {:.6f}, BCE loss: {:.6f}'.format(reg_loss, bceloss))
        torch.save(encoder, "encoder{}".format(epoch))
        torch.save(dc, "dc{}".format(epoch))
        
        AUC = run_epoch(dev_data, False, encoder, dc, optimizer_encoder, optimizer_dc, 5)
        print('Val AUC: {:.6f}'.format(AUC))
        
        print('This epoch took: {:.6f}'.format(time.time() - lasttime))
        lasttime = time.time()

        
def run_epoch(data, is_training, encoder, dc, optimizer_encoder, optimizer_dc, batch_size):
    '''
    Train model for one pass of train data, and return loss, acccuracy
    '''
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        drop_last=False)

    losses = []
    bcelosses = []
    targets = []
    scores = []

    if is_training:
        encoder.train()
        dc.train()
    else:
        encoder.eval()
        dc.eval()

    for batch in data_loader:
        pid_title = torch.unsqueeze(Variable(batch['pid_title']), 1)
        pid_body = torch.unsqueeze(Variable(batch['pid_body']), 1)
        rest_title = Variable(batch['rest_title'])
        rest_body = Variable(batch['rest_body'])
        
        if is_training:
            android_title = torch.unsqueeze(Variable(batch['android_title']), 1).cuda()
            android_body = torch.unsqueeze(Variable(batch['android_body']), 1).cuda()
        
        pid_title, pid_body = pid_title.cuda(), pid_body.cuda()
        rest_title, rest_body = rest_title.cuda(), rest_body.cuda()
        
        true_batch_size = pid_title.size()[0]
        
        if is_training:
            optimizer_encoder.zero_grad()
            optimizer_dc.zero_grad()
        
        pt = encoder(pid_title)
        pb = encoder(pid_body)
        rt = encoder(rest_title)
        rb = encoder(rest_body)
        
        pid_tensor = (pt + pb)/2
        rest_tensor = (rt + rb)/2
        
        if is_training:
            android_tensor = (encoder(android_title) + encoder(android_body))/2
            dropout = nn.Dropout(p = 0.2)
            
            pid_tensor = dropout(pid_tensor)
            rest_tensor = dropout(rest_tensor)
            android_tensor = dropout(android_tensor)
            
            pid_domain = dc(pid_tensor.view(true_batch_size, -1))
            android_domain = dc(android_tensor.view(true_batch_size, -1))
            
            # vectors are of dim batch_size x 2
            
            softmax = nn.Softmax()
            pid_prob = softmax(pid_domain)
            android_prob = softmax(android_domain)
            probs = torch.cat((pid_prob, android_prob)).clamp(min = 1e-5, max = 1 - 1e-5)
            target = Variable(torch.FloatTensor([0] * true_batch_size + [1] * true_batch_size)).cuda()
            dc_loss = torch.nn.BCELoss()
            
            lmbda = 1e-2
            bceloss = dc_loss(probs[:,0], target)
            loss = loss_function(pid_tensor, rest_tensor, margin = 1.0) - lmbda * bceloss
            
            loss.backward()
            losses.append(loss.cpu().data[0])
            bcelosses.append(bceloss.cpu().data[0])
            optimizer_encoder.step()
            optimizer_dc.step()
        else:
            expanded = pid_tensor.expand_as(rest_tensor)
            similarity = cs(expanded, rest_tensor, dim=2).squeeze(2)
            similarity = torch.FloatTensor(similarity.data.cpu().numpy())
            labels = batch['labels']
            
            for sim in similarity:
                scores.append(sim)
            targets.extend(labels.view(-1))

    # Calculate epoch level scores
    if is_training:
        avg_loss = np.mean(losses)
        avg_bce = np.mean(bcelosses)
        return (avg_loss, avg_bce)
    else:
        aucmeter = AUCMeter()
        aucmeter.reset()
        
        output = torch.cat(scores)
        expect = torch.LongTensor(targets)
        aucmeter.add(output, expect)
        return aucmeter.value(max_fpr=0.05)

In [8]:
z = train_model(train_ex, eval_set, encoder, dc)

-------------
Epoch 1:

Train loss: 1.019697, BCE loss: 0.551535
Val AUC: 0.448698
This epoch took: 112.701718
-------------
Epoch 2:

Train loss: 0.977446, BCE loss: 0.451958
Val AUC: 0.625105
This epoch took: 113.998625
-------------
Epoch 3:

Train loss: 0.922405, BCE loss: 0.447932
Val AUC: 0.631576
This epoch took: 115.038333
-------------
Epoch 4:

Train loss: 0.900064, BCE loss: 0.435734
Val AUC: 0.652776
This epoch took: 114.114070
-------------
Epoch 5:

Train loss: 0.884822, BCE loss: 0.418224
Val AUC: 0.673163
This epoch took: 113.325081
-------------
Epoch 6:

Train loss: 0.875227, BCE loss: 0.380646
Val AUC: 0.673729
This epoch took: 113.280634
-------------
Epoch 7:

Train loss: 0.864080, BCE loss: 0.354999
Val AUC: 0.676257
This epoch took: 113.217903
-------------
Epoch 8:

Train loss: 0.856293, BCE loss: 0.334090
Val AUC: 0.678822
This epoch took: 113.171992
-------------
Epoch 9:

Train loss: 0.848566, BCE loss: 0.318023
Val AUC: 0.665679
This epoch took: 113.194482
-

Process Process-154:
Process Process-156:
Process Process-155:
Process Process-153:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/sunl/.conda/envs/my_pytorch/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_

KeyboardInterrupt: 

In [9]:
encoder = torch.load("encoder8")
dc = torch.load("dc8")
z = run_epoch(eval_set, False, encoder, dc, None, None, 5)
print(z)

0.678822254347


In [10]:
encoder = torch.load("encoder8")
dc = torch.load("dc8")
eval_path = "../data_Android/test_android.txt"
eval_anno = dr.read_annotations(eval_path, K_neg = -1, prune_pos_cnt = -1, ignore_dup = True)
eval_set = dr.create_dev_set(ids_corpus, eval_anno)

z = run_epoch(eval_set, False, encoder, dc, None, None, 5)
print(z)

0.672892484542
